In [ ]:
%cd /home/jovyan/onceuponai/websites/src/stories-fantasy-shop-assistant/notebooks
from IPython.core.display import HTML
HTML("""
<link rel="stylesheet" type="text/css" href="//fonts.googleapis.com/css?family=Bangers" />
<style>
h1 {font-family: Bangers !important; font-size: 150px !important}
h2 {font-family: Bangers !important; font-size: 120px !important}
h3 {font-family: Bangers !important; font-size: 100px !important}
h4 {font-family: Bangers !important; font-size: 80px !important}
h5 {font-family: Bangers !important; font-size: 50px !important}
h6 {font-family: Bangers !important; font-size: 30px !important}
.txt7 {font-family: Bangers !important; font-size: 70px !important}
.txt6 {font-family: Bangers !important; font-size: 60px !important}
.txt5 {font-family: Bangers !important; font-size: 50px !important}
.txt4 {font-family: Bangers !important; font-size: 40px !important}
.txt3 {font-family: Bangers !important; font-size: 30px !important}
.cap {font-family: Bangers !important; font-size: 30px !important}
.controls
{
    display: none !important;
}

.slide-number
{
    display: none !important;
}
.inline-block {
   display: inline-block;
}

.backgrounds {
   background-color: #fff !important;
}

.controls {
  display: none !important;
}
</style>
""")

import pandas as pd
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install deltalake polars

# Prepare data

In [ ]:
import yaml
from deltalake.writer import write_deltalake
import pandas as pd
import polars as pl

data = yaml.safe_load(open("recipes.yaml", "r"))

prep={}


df = pl.DataFrame(data["Recipes"])
df.to_pandas()

# Embeddings

In [ ]:
import adventures

a = adventures.Adventures("intfloat/multilingual-e5-small")

In [ ]:
a.embeddings(["witaj przygodo"])

In [ ]:
items = []
for combined_text in df['Recipe']:
    emb = a.embeddings([combined_text])
    items.append({"item": combined_text, "vector": emb[0]})

items

# Fake words

In [ ]:
import random

subjects = [
    "The mischievous squirrel", "The colorful birds", "The playful puppies", 
    "The dedicated teacher", "The curious boy", "The brilliant inventor", "Girl", "Cat", "Mouse", "Dog"
]

verbs = [
    "danced", "explored", "built", "discovered", "imagined", 
    "sang", "cooked", "programmed","run", "sing", "walk", 
]

objects = ["the sandwich", "a brilliant app", "over the fence", "a beautiful song", "in the sky"]

for i in range(300):
    phrase = random.choice(subjects) + " " + random.choice(verbs) + " " + random.choice(objects)
    emb = a.embeddings([phrase])
    items.append({"item": phrase, "vector": emb[0]})

# Lance DB

In [ ]:
import os
import getpass

print("ACCOUNT NAME")
os.environ["AZURE_STORAGE_ACCOUNT_NAME"]=getpass.getpass()

print("ACCOUNT KEY")
os.environ["AZURE_STORAGE_ACCOUNT_KEY"]=getpass.getpass()

In [ ]:
import getpass
import lancedb

db = await lancedb.connect_async("az://recipesv2/vectors")
tbl = await db.create_table("recipes_vectors", data=items)

In [ ]:
res = await tbl.create_index("vector")

# Search

In [ ]:
import lancedb

emb = a.embeddings(["Chciałbym zrobić żurek"])

tbl = await db.open_table("recipes_vectors")
res = await tbl.query().nearest_to(emb[0]).limit(1).to_pandas()
res